# Encontro 03: Grafos Reais

Importando a biblioteca:

In [1]:
import sys
sys.path.append('..')

import socnet as sn

Carregando e visualizando o grafo:

In [2]:

sn.node_size = 3
sn.node_color = (0, 0, 0)
sn.edge_width = 1
sn.edge_color = (192, 192, 192)
sn.node_label_position = 'top center'

g = sn.load_graph('mySpotify.gml')

sn.show_graph(g, nlab=True)

Dependendo do tamanho, o grafo pode demorar um pouco para aparecer.

In [4]:
def simulate_betweenness_transfer_geodesic(g):

    # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_betweenness'] = 0

    for _ in range(TIMES):

        # Inicialização de todos os betweenness.
        for n in g.nodes():
            g.node[n]['betweenness'] = 0

        for s in g.nodes():
            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
                        for n in g.nodes():
                            g.node[n]['owner'] = False
                        g.node[s]['owner'] = True

                        for n in g.nodes():
                            if n != s and n != t:
                                g.node[n]['partial_betweenness'] = 0

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()

                            for n in g.nodes():
                                if g.node[n]['owner']:

                                    # TRANSFERÊNCIA: Escolhemos aleatoriamente um dos vizinhos válidos.
                                    # GEODÉSICA: Os vizinhos válidos são os que pertencem a geodésicas.
                                    m = choice(g.node[n]['shortest_neighbors'])
                                    nodes_reached.add(m)

                                    # TRANSFERÊNCIA: O fluxo transfere o bem para os nós que o fluxo
                                    # alcança, portanto o nó deixa de ser dono. Nos processos baseados
                                    # em duplicação, a linha abaixo não pode ser executada.
                                    g.node[n]['owner'] = False

                            # Todos os nós que o fluxo alcança tornam-se donos.
                            for n in nodes_reached:
                                g.node[n]['owner'] = True

                            # Se alcançamos t, interrompemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo e tentamos novamente.
                            if not nodes_reached:
                                break

                            for n in nodes_reached:
                                if n != s and n != t:
                                    g.node[n]['partial_betweenness'] += 1

                    # Soma de todos os betweenness parciais dos intermediários.
                    for n in g.nodes():
                        if n != s and n != t:
                            g.node[n]['betweenness'] += g.node[n]['partial_betweenness']

        # Incremento das médias. Divide-se o valor por 2 para
        # desconsiderar a simetria de um grafo não-dirigido.
        for n in g.nodes():
            g.node[n]['simulated_abetweenness'] += g.node[n]['betweenness'] / 2

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_betweenness'] /= TIMES

In [5]:
from random import choice

TIMES = 1000

In [6]:
sn.build_betweenness(g)

result = []
for n in g.nodes():
    result.append((g.node[n]['label'], g.node[n]['theoretical_betweenness']))
from operator import itemgetter
print(sorted(result,key=itemgetter(1), reverse=True))
    

[('Grupo Fundo De Quintal', 2374.0133769459962), ('Katinguele', 1968.4355280226287), ('Monobloco', 1443.339675426872), ('Olodum', 1409.2722964019183), ('Sem Compromisso', 1010.4465776163477), ('Reinaldo', 804.4585444991335), ('Benito Di Paula', 629.6438997854066), ('Agepe', 572.6673216990596), ('Fabio Jr.', 512.553897769795), ('Os Travessos', 497.24749609781327), ('Alcione', 473.65593654349027), ('Jorge Aragao', 463.0582008641768), ('Arlindo Cruz', 452.4444952822195), ('Daniela Mercury', 440.08255662384323), ('Sandra De Sa', 377.5901380774768), ('So Pra Contrariar', 350.714813616965), ('Almir Guineto', 293.0584759277254), ('Guilherme Arantes', 275.79230210896844), ('Imaginasamba', 245.7388584521637), ('Soweto', 231.0732730455176), ('Dudu Nobre', 203.0602551309483), ('Swing & Simpatia', 176.6412292123072), ('Samprazer', 139.1618349962041), ('Ara Ketu', 137.4795130164775), ('Pixote', 129.7758129865308), ('Bezerra Da Silva', 121.14611083937311), ('Pique Novo', 114.77428037329608), ('Oswal

In [ ]:
result_degree = []
for n in g.nodes():
    result_degree.append((g.node[n]['label'], g.degree(n)))
from operator import itemgetter
print(sorted(result_degree,key=itemgetter(1), reverse=True))